In [35]:
import pandas as pd
import openpyxl as opxl
from googleapiclient.discovery import build
from IPython.display import JSON
import isodate

In [3]:
API_KEY = ''

In [4]:
channel_id = ['UC2pmfLm7iq6Ov1UwYrWYkZA']

In [5]:
api_service_name = "youtube"
api_version = "v3"
    # Get credentials and create an API client
youtube = build(api_service_name, api_version, developerKey = API_KEY)

In [6]:
def get_channel_info(youtube, channel_id):
    
    all_data = []

    request = youtube.channels().list(
    part="snippet,contentDetails,statistics",
    id=','.join(channel_id)
    )
    response = request.execute()

    for item in response['items']:
        data = {'channelName':item['snippet']['title'],
                'subscibres': item['statistics']['subscriberCount'],
                'views': item['statistics']['viewCount'],
                'totalViews': item['statistics']['videoCount'],
                'playlistId': item['contentDetails']['relatedPlaylists']['uploads']}
        all_data.append(data)

    return pd.DataFrame(all_data)

In [7]:
channel_stats = get_channel_info(youtube, channel_id)

In [8]:
channel_stats

,channelName,subscibres,views,totalViews,playlistId
0,Vevo,20000000,561095032,1536,UU2pmfLm7iq6Ov1UwYrWYkZA


In [9]:
playlist_id = "UU2pmfLm7iq6Ov1UwYrWYkZA"

def get_video_ids(youtube,playlist_id):

    video_ids = []

    request = youtube.playlistItems().list(
        part="snippet,contentDetails",
        playlistId = playlist_id,
        maxResults = 50
    )
    response = request.execute()

    for item in response['items']:
        video_ids.append(item['contentDetails']['videoId'])

    next_page_token = response.get('nextPageToken')
    while next_page_token is not None:
        request = youtube.playlistItems().list(
        part="snippet,contentDetails",
        playlistId = playlist_id,
        maxResults = 50,
        pageToken = next_page_token)
        response = request.execute()

        for item in response['items']:
            video_ids.append(item['contentDetails']['videoId'])

        next_page_token = response.get('nextPageToken')

    return video_ids

In [10]:
video_ids = get_video_ids(youtube,playlist_id)

In [11]:
len(video_ids)

1536

In [12]:
def get_video_details(youtube, video_ids):

    all_video_info = []

    for i in range(0, len(video_ids),50):

        request = youtube.videos().list(
            part="snippet,contentDetails,statistics",
            id = ','.join(video_ids[i:i+50])
        )
        response = request.execute()

        JSON(response)

        for video in response['items']:
            stats_to_keep = {'snippet': ['channelTitle', 'title', 'description', 'tags', 'publishedAt'],
                            'statistics': ['viewCount', 'likeCount', 'favouriteCount', 'commentCount'],
                            'contentDetails': ['duration', 'definition', 'caption']}
            video_info = {}
            video_info['video_id'] = video['id']

            for k in stats_to_keep.keys():
                for v in stats_to_keep[k]:
                    try:
                        video_info[v] = video[k][v]
                    except:
                        video_info[v] = None

            all_video_info.append(video_info)

    return pd.DataFrame(all_video_info)

In [24]:
video_df = get_video_details(youtube, video_ids)

video_df

,video_id,channelTitle,title,description,tags,publishedAt,viewCount,likeCount,favouriteCount,commentCount,duration,definition,caption
0,yh6WxdKr94A,Vevo,"Hip-Hop Video Weekly: Doja Cat ""Paint The Town...",Our latest Hip-Hop Video Weekly round-up featu...,None,2023-08-07T23:30:11Z,4630,178,None,9,PT15S,hd,false
1,zUfOsfBgiOk,Vevo,"Global Video of the Week: Halle ""Angel""",Our latest pick for Global Video of the Week i...,None,2023-08-07T20:26:19Z,2616,147,None,5,PT15S,hd,false
2,l_48xuKTbCM,Vevo,Watch Khalid Set The Mood,Can you guess @khalid's go-to karaoke song? Se...,None,2023-08-02T17:17:15Z,5624,193,None,22,PT19S,hd,false
3,s9dAmXl4xJA,Vevo,"Global Video of the Week: @jessieware, @roisin...",,None,2023-07-31T21:36:34Z,5068,158,None,3,PT15S,hd,false
4,dTimjJ86Khc,Vevo,"Hip-Hop Video Weekly | Ice Spice ""Deli""","​@IceSpice, @TravisScottXX, @ASAPROCKYUPTOWN...",None,2023-07-31T21:32:31Z,8856,214,None,7,PT15S,hd,false
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1531,nbRw70D0Zvc,Vevo,Various Artists - We Are The World 25 for Hait...,Music video by Various Artists performing We A...,"[Various, Artists, Sony, Music, Entertainment]",2010-02-12T20:54:01Z,264628,614,None,40,PT44S,sd,false
1532,6oXtClr0aKs,Vevo,Various Artists - We Are The World 25 for Hait...,Music video by Various Artists performing We A...,"[Various, Artists, Sony, Music, Entertainment]",2010-02-12T20:31:45Z,289235,799,None,54,PT1M6S,sd,false
1533,dMo8JuPPb0M,Vevo,Adam Lambert - Adam Lambert's VEVO Hotel Party,Music video by Adam Lambert performing Adam La...,"[Adam, Lambert, Epic, Pop]",2010-01-15T19:36:26Z,539430,2624,None,566,PT1M46S,hd,false
1534,TjmRxdkLmsk,Vevo,Mariah Carey - Mariah Carey Sings H.A.T.E.U. f...,Music video by Mariah Carey performing Mariah ...,"[Mariah, Carey, Epic, Pop]",2010-01-15T16:54:38Z,380371,4510,None,208,PT32S,hd,false


# Data Pre-processing

In [14]:
video_df.isnull().any()

video_id          False
channelTitle      False
title             False
description       False
tags               True
publishedAt       False
viewCount         False
likeCount          True
favouriteCount     True
commentCount       True
duration          False
definition        False
caption           False
dtype: bool

In [15]:
video_df.dtypes

video_id          object
channelTitle      object
title             object
description       object
tags              object
publishedAt       object
viewCount         object
likeCount         object
favouriteCount    object
commentCount      object
duration          object
definition        object
caption           object
dtype: object

In [25]:
numeric_cols = ['viewCount', 'likeCount', 'favouriteCount', 'commentCount']
video_df[numeric_cols] = video_df[numeric_cols].apply(pd.to_numeric, errors = 'coerce', axis = 1)

In [54]:
video_df['publishedAt'] = pd.to_datetime(video_df['publishedAt'], format='%d %B %Y').dt.date


video_df['publishedAt']

0       2023-08-07
1       2023-08-07
2       2023-08-02
3       2023-07-31
4       2023-07-31
           ...    
1531    2010-02-12
1532    2010-02-12
1533    2010-01-15
1534    2010-01-15
1535    2009-12-31
Name: publishedAt, Length: 1536, dtype: object

In [ ]:
video_df['durationSecs'] = video_df['duration'].apply(lambda x: isodate.parse_duration(x))

In [43]:
video_df['durationSecs'] = video_df['durationSecs'].astype('timedelta64[s]')
video_df['durationSecs']

0      0 days 00:00:15
1      0 days 00:00:15
2      0 days 00:00:19
3      0 days 00:00:15
4      0 days 00:00:15
             ...      
1531   0 days 00:00:44
1532   0 days 00:01:06
1533   0 days 00:01:46
1534   0 days 00:00:32
1535   0 days 00:03:29
Name: durationSecs, Length: 1536, dtype: timedelta64[s]

In [32]:
video_df['tagCount'] = video_df['tags'].apply(lambda x: 0 if x is None else len(x))

In [56]:

ruta = ''
nombre_archivo = 'datosYoutube.xlsx'
ruta_archivo = ruta + nombre_archivo


video_df.to_excel(ruta_archivo, index = False)